In [1]:
#import necesary libraries
import pandas as pd
import io
import numpy as np
import seaborn as sns
import earthpy as et

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Use white grid plot background from seaborn
sns.set(font_scale=1.5, style="whitegrid")
import matplotlib.pyplot as plt


#simulated
data1 = pd.read_csv(r'D:\OneDrive\OneDrive - WFT\Compartido\Well_Datasets\La_Calera_Pluspetrol\Post_Process\sim_logs\outputs\Last_C1\Log14042021.csv')

simulation = 'PVT11'

print("Data Loaded OK")




Data Loaded OK


In [2]:
c = s = st = 0

for x in data1['Pressure[Bar]']:
    c = c +1
    if x == 3005:
        s = s + 1
        if s == 1:
            st = c
        
#print(data1.loc[st:st+11])

data1['WELL Name'] = ''
well = ['LCA-3005', 'LCA-3006', 'LCA-3007','LCA-3008', 'LCA-3009', 'LCA-3010','LCA-3011', 'LCA-3012', 'LCA-3013','LCA-3083','LCA-3084','LCA-3085']
#data1['WELL Name'].loc[st:st+11] = 'LCA-3005'
r = 0
for w in well:
    data1['WELL Name'].loc[st+(r*13):st+(r*13)+13] = w
    r = r+1
    #print(w)
    

#data1.loc[st:st+15]
#data1.columns

data1[['WELL Name','Time', 'Pressure[Bar]', 'DP[Bar]', 'Temperature[C]',
       'Velocity[m/s]', 'Quality',
       'WaterCut[%]', 'Qg[m3/s]', 'Ql[m3/s]', 'Qo[m3/s]',
       'Qw[m3/s]', 'QgStd[m3/s]', 'QlStd[m3/s]', 'QoStd[m3/s]', 'QwStd[m3/s]',
       'WWC[%]', 'TotGOR', 'GVF[%]','GasDen[kg/m3]','GasVisc[cP]', 
       'LiqDen[kg/m3]', 'LiqVisc[cP]', 
       'PVT_Var', 'MVT_Var_out', 'Sonar_Vel_Var', 
       'PI_Status_Out', 'ErrQt', 'Vm[m/s]', 
       'diffPress_Input[bar]', 'diffPressZero[bar]', 'ParamFile', 'PVTFile',
       'Rhog', 'Rhol', 'Rhoo', 'Rhow'
       ]].info()


#filetering columns

ds_Q = data1[['WELL Name','Time', 'Pressure[Bar]', 'DP[Bar]', 'Temperature[C]',
       'Velocity[m/s]', 'Quality',
       'WaterCut[%]', 'Qg[m3/s]', 'Ql[m3/s]', 'Qo[m3/s]',
       'Qw[m3/s]', 'QgStd[m3/s]', 'QlStd[m3/s]', 'QoStd[m3/s]', 'QwStd[m3/s]',
       'WWC[%]', 'TotGOR', 'GVF[%]','GasDen[kg/m3]','GasVisc[cP]', 
       'LiqDen[kg/m3]', 'LiqVisc[cP]', 
       'PVT_Var', 'MVT_Var_out', 'Sonar_Vel_Var', 
       'PI_Status_Out', 'ErrQt', 'Vm[m/s]', 
       'diffPress_Input[bar]', 'diffPressZero[bar]', 'ParamFile', 'PVTFile',
       'Rhog', 'Rhol', 'Rhoo', 'Rhow'
       ]]

#adding columns with 24hs rates

ds_Q.loc[:,"Qg[m3/d]"] = ds_Q.loc[:,"Qg[m3/s]"] * 86400
ds_Q.loc[:,"Ql[m3/d]"] = ds_Q.loc[:,"Ql[m3/s]"] * 86400
ds_Q.loc[:,"Qo[m3/d]"] = ds_Q.loc[:,"Qo[m3/s]"] * 86400
ds_Q.loc[:,"Qw[m3/d]"] = ds_Q.loc[:,"Qw[m3/s]"] * 86400
ds_Q.loc[:,"Gas Rate Sm3/d"] = ds_Q.loc[:,"QgStd[m3/s]"] * 86400
ds_Q.loc[:,"QlStd[m3/d]"] = ds_Q.loc[:,"QlStd[m3/s]"] * 86400
ds_Q.loc[:,"Oil Flow Sm3/d"] = ds_Q.loc[:,"QoStd[m3/s]"] * 86400
ds_Q.loc[:,"Water Flow Rate m3/d"] = ds_Q.loc[:,"QwStd[m3/s]"] * 86400

#cleaning columns

to_drop = ['Qg[m3/s]', 'Ql[m3/s]', 'Qo[m3/s]',
       'Qw[m3/s]', 'QgStd[m3/s]', 'QlStd[m3/s]', 'QoStd[m3/s]', 'QwStd[m3/s]']

ds_Qf= ds_Q.drop(to_drop, axis=1)

ds = ds_Qf.copy()
#ds = ds.set_index('Time')
#ds2 = ds_Qf.set_index('Time').groupby(pd.Grouper(freq='17min')).mean()

#ds.index = pd.to_datetime(ds.index)


ds['DATE & TIME'] = ds['Time']
ds['Test'] = simulation
ds['Gas Spec. Gravity air=1'] = ''
ds['Well Head Pressure  psig'] =''
ds['Well Head Temp deg C']=''
ds['Sep. Static Pressure psig']= ds.loc[:,'Pressure[Bar]'] * 14.5038
ds['Sep. Gas Temp deg C'] = ds.loc[:,'Temperature[C]']
ds['CHOKE'] = '40'
ds['Gas Cumm. Sm3'] = ''
ds['GOR (Gas/Oil Ratio) m3/m3'] = ds['TotGOR']
ds['Oil Density  g/cm3'] = ''
ds['Oil Temp deg C'] = ''
ds[' Raw Oil Flow  m3/d'] = ds['Qo[m3/d]']
ds['WaterDensity Kg/m3'] = ''
ds['Flow Water Cut  %'] = ds['WWC[%]']

d = ds[['DATE & TIME','CHOKE','WELL Name', 'Test', 'Well Head Pressure  psig', 'Well Head Temp deg C',
         'Sep. Static Pressure psig', 'Sep. Gas Temp deg C', 'Gas Spec. Gravity air=1','Gas Rate Sm3/d',
        'Gas Cumm. Sm3','GOR (Gas/Oil Ratio) m3/m3','Oil Density  g/cm3','Oil Temp deg C',
         ' Raw Oil Flow  m3/d','Oil Flow Sm3/d','Water Flow Rate m3/d','WaterDensity Kg/m3',
        'Flow Water Cut  %',
        'DP[Bar]', 'Temperature[C]',
       'Velocity[m/s]', 'Quality',       
       'GVF[%]','GasDen[kg/m3]','GasVisc[cP]', 
       'LiqDen[kg/m3]', 'LiqVisc[cP]','Qg[m3/d]',"Ql[m3/d]","Qo[m3/d]"
        ]]

d = d[d['WELL Name']!='']
d = d[d['Gas Rate Sm3/d']!=0]

d.to_csv(r'D:\OneDrive\OneDrive - WFT\Compartido\Well_Datasets\La_Calera_Pluspetrol\Post_Process\append\sim'+simulation+'.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 37 columns):
WELL Name               1567 non-null object
Time                    1567 non-null object
Pressure[Bar]           1567 non-null float64
DP[Bar]                 1567 non-null float64
Temperature[C]          1567 non-null float64
Velocity[m/s]           1567 non-null float64
Quality                 1567 non-null float64
WaterCut[%]             1567 non-null int64
Qg[m3/s]                1567 non-null float64
Ql[m3/s]                1567 non-null float64
Qo[m3/s]                1567 non-null float64
Qw[m3/s]                1567 non-null float64
QgStd[m3/s]             1567 non-null float64
QlStd[m3/s]             1567 non-null float64
QoStd[m3/s]             1567 non-null float64
QwStd[m3/s]             1567 non-null float64
WWC[%]                  1567 non-null float64
TotGOR                  1567 non-null float64
GVF[%]                  1446 non-null float64
GasDen[kg/m3]          

C:\Users\e196315\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\e196315\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\e196315\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

Export to next step

In [3]:
#TO combine multiple files

import glob
import pandas as pd

# get data file names
path = r'D:\OneDrive\OneDrive - WFT\Compartido\Well_Datasets\La_Calera_Pluspetrol\Post_Process\append'
filenames = glob.glob(path + "\*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename,low_memory=False).assign(filename = filename))

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True,sort=False)

big_frame.to_csv(r"D:\OneDrive\OneDrive - WFT\Compartido\Well_Datasets\La_Calera_Pluspetrol\Post_Process\append\out\raw_full.csv", index=False, encoding='utf-8-sig')